In [ ]:
import cv2
import numpy as np
import plotly.graph_objects as go
import plotly.offline as pyo

import matplotlib.pyplot as plt
import matplotlib.animation as animation

from PIL import Image, ImageOps
from plotly.subplots import make_subplots
from os import getcwd
from math import ceil

%matplotlib inline

In [ ]:
def freq_filter(x_freq, y_freq, factor=2.4):
    eps = 10 ** -8
    x, y = np.meshgrid(x_freq, y_freq)
    f = np.hypot(x, y)
    f = f ** factor + eps
    return normalize(1 / f)


def freq_sharp_round_filter(x_freq, y_freq, radius, reverse=False):
    eps = 10 ** -8
    x, y = np.meshgrid(x_freq, y_freq)
    if reverse:
        f = np.zeros((x_size, y_size))
    else:
        f = np.ones((x_size, y_size))
    
    for i, xx in enumerate(x_freq):
        for j, yy in enumerate(y_freq):
            if xx ** 2 + yy ** 2 <= radius ** 2:
                if reverse:
                    f[i, j] = 1
                else:
                    f[i, j] = 0
    return normalize(f)


def freq_sharp_square_filter(x_freq, y_freq, width, reverse=False):
    eps = 10 ** -8
    x, y = np.meshgrid(x_freq, y_freq)
    if reverse:
        f = np.zeros((x_size, y_size))
    else:
        f = np.ones((x_size, y_size))
    
    for i, xx in enumerate(x_freq):
        for j, yy in enumerate(y_freq):
            if abs(xx) + abs(yy) <= width:
                if reverse:
                    f[i, j] = 1
                else:
                    f[i, j] = 0
    return normalize(f)


def spatial_smooth_filter(x_size, y_size, depth, horiz=True):
    values = np.linspace(0, 1, depth)
    values = 6 * values ** 5 - 15 * values ** 4 + 10 * values ** 3
    values = 1 - values
    if horiz:
        kernel = np.tile(values, (y_size, 1))
    else:
        kernel = values[:, np.newaxis] * np.ones((1, x_size))   
    return kernel


def find_ft(img):
    ft = np.fft.fft2(img)
    return np.fft.fftshift(ft)


def find_ift(ft):
    ift = np.fft.ifftshift(ft)
    return np.fft.ifft2(ift)


def normalize(arr):
    min_val = abs(np.min(arr))
    max_val = abs(np.max(arr))
    return (arr + min_val) / (min_val + max_val)


def normalize_img(img):
    img_min = abs(np.min(img))
    img_max = abs(np.max(img))
    img_norm = 255 * (img + img_min) / (img_min + img_max)
    return img_norm.astype(int)


def gen_cloud(x_size, y_size, factor=2.4):
    xx = np.linspace(-x_size / 2, x_size / 2, x_size)
    yy = np.linspace(-y_size / 2, y_size / 2, y_size)
    whitenoise = np.random.normal(0, 1, (y_size, x_size))
    cloud_freq = find_ft(whitenoise)
    kernel = freq_filter(xx, yy, factor=factor)
    cloud_freq_filtered = cloud_freq * kernel
    cloud_spatial = find_ift(cloud_freq_filtered).real
    return normalize_img(cloud_spatial)


def show_images(*images, vmin=0, vmax=255, x_fig_size=10, cmap='gray', y_fig_size=10, graphs_per_row=2):
    if len(images) == 1:
        plt.imshow(images[0], cmap='gray', vmin=0, vmax=255)
    else:
        row_num = ceil(len(images) / graphs_per_row)
        col_num = ceil(len(images) / row_num)
    
        f, axes = plt.subplots(row_num, col_num, sharey=True, figsize=(x_fig_size, y_fig_size))

        for ax, img in zip(axes.flatten(), images):
            ax.imshow(img, cmap='gray', vmin=0, vmax=255)
        
        
def make_img_transition_x(img, depth, is_dx_pos=True):
    y_size, x_size = img.shape
    additional_img = gen_cloud(x_size + depth, y_size)   
    transition_kernel = spatial_smooth_filter(x_size, y_size, depth)     
    
    new_img = np.copy(img)
    if is_dx_pos:
        new_img[:, -depth:x_size] = img[:, -depth:x_size] * transition_kernel + \
                                additional_img[:, 0:depth] * (1 - transition_kernel)
        return new_img, additional_img[:, depth:]    
    else:
        transition_kernel = np.fliplr(transition_kernel)
        new_img[:, 0:depth] = img[:, 0:depth] * transition_kernel + \
                          additional_img[:, -depth:] * (1 - transition_kernel)  
        return new_img, additional_img[:, 0:-depth]    


def make_img_transition_y(img, depth, is_dy_pos=True):
    y_size, x_size = img.shape
    additional_img = gen_cloud(x_size, y_size + depth)   
    transition_kernel = spatial_smooth_filter(x_size, y_size, depth, horiz=False)
        
    new_img = np.copy(img)
    if is_dy_pos:
        new_img[-depth:x_size, :] = img[-depth:x_size, :] * transition_kernel + \
                                additional_img[0:depth, :] * (1 - transition_kernel)
        return new_img, additional_img[depth:, :]    
    else:
        transition_kernel = np.flipud(transition_kernel)
        new_img[0:depth, :] = img[0:depth, :] * transition_kernel + \
                          additional_img[-depth:, :] * (1 - transition_kernel)  
        return new_img, additional_img[0:-depth:1, :]    
    

def make_img_transition_xy(img, img_depth, is_dx_pos=True, is_dy_pos=True):
    new_img, add_img = make_img_transition_x(img, depth, is_dx_pos=is_dx_pos)
    new_img = np.concatenate((new_img, add_img), axis=1)
    
    new_img, add_img = make_img_transition_y(new_img, depth, is_dy_pos=is_dy_pos)
    new_img = np.concatenate((new_img, add_img), axis=0)
    
    return new_img


def shift_img_x(img, width, dx, is_dx_pos=True): 
    # Wind blows in negative X dirrection
    if is_dx_pos:
        return img[:, dx:width + dx]
    else:
        return img[:, width - dx:-dx]


def shift_img_y(img, height, dy, is_dy_pos=True):
    # Wind blows in negative Y dirrection
    if is_dy_pos:
        return img[dy:height + dy, :]
    else:
        return img[height - dy:-dy, :]

    
def shift_img_xy(img, window_shape, dx, dy, is_dx_pos=True, is_dy_pos=True):
    height, width = window_shape
    
    shifted_img = shift_img_x(new_img, width, dx, is_dx_pos=is_dx_pos)
    shifted_img = shift_img_y(shifted_img, height, dy, is_dy_pos=is_dy_pos)
    
    return shifted_img


CURRENT_DIR = getcwd()

# Rubik's cube example

In [ ]:
cube = Image.open(CURRENT_DIR + '/img/cube_1.png').convert('L')
cube_fr = find_ft(cube)

x_size, y_size = cube_fr.shape
xx = np.linspace(-x_size / 2, x_size / 2, x_size)
yy = np.linspace(-y_size / 2, y_size / 2, y_size)

kernel = normalize(freq_sharp_round_filter(yy, xx, radius=50))
eps = 10 ** -10

f, (ax1, ax2) = plt.subplots(1, 2, sharey=True, figsize=(10, 10))
ax1.imshow(cube, cmap='gray', vmin=0, vmax=255)
# Wrong scaling, but it gives a really interesting visual representation
ax2.imshow(find_ift(cube_fr * kernel).real, cmap='gray', vmin=0, vmax=np.max(np.log(np.absolute(cube_fr + eps))))

In [ ]:
cube = Image.open(CURRENT_DIR + '/img/woman.jpeg').convert('L')
cube_fr = find_ft(cube)

x_size, y_size = cube_fr.shape
xx = np.linspace(-x_size / 2, x_size / 2, x_size)
yy = np.linspace(-y_size / 2, y_size / 2, y_size)

fps = 30
seconds_num = 5
x_mesh, y_mesh = np.meshgrid(xx, yy)

radius = 300
cube_freq = [cube]
kernel_arr = []

for i in range(fps * seconds_num):
    kernel = normalize(freq_sharp_round_filter(xx, yy, radius=i / fps / seconds_num * 150))
    kernel_arr.append(kernel)
    cube_freq.append(normalize_img(find_ift(cube_fr * kernel).real))

In [ ]:
%matplotlib notebook

fig = plt.figure(figsize=(8,8))
im = plt.imshow(cube_freq[0], interpolation='none', aspect='equal', cmap='gray')


def animate_func(i): 
    im.set_array(cube_freq[i])
    return [im]


anim = animation.FuncAnimation(
    fig, 
    animate_func, 
    frames = seconds_num * fps,
    interval = 1000 / fps, # in ms
)


# anim.save('women.gif', writer='pillow')

In [ ]:
%matplotlib inline

cube = Image.open(CURRENT_DIR + '/img/cube_1.png').convert('L')
cube_fr = find_ft(cube)

eps = 10 ** -10

f, (ax1, ax2) = plt.subplots(1, 2, sharey=True, figsize=(10, 10))
ax1.imshow(cube, cmap='gray', vmin=0, vmax=255)
ax2.imshow(np.log(np.absolute(cube_fr + eps)), cmap='gray')

# Phase correlation example

In [ ]:
f1 = Image.open(CURRENT_DIR + '/img/horse_1.png').convert('L')
f2 = Image.open(CURRENT_DIR + '/img/horse_translated_1.png').convert('L')

f1_freq = find_ft(f1)
f2_freq = find_ft(f2)

f, (ax1, ax2) = plt.subplots(1, 2, sharey=True, figsize=(15, 15))
ax1.imshow(f1, cmap='gray', vmin=0, vmax=255)
ax2.imshow(f2, cmap='gray', vmin=0, vmax=255)

In [ ]:
f, (ax1, ax2) = plt.subplots(1, 2, sharey=True, figsize=(15, 15))
ax1.imshow(np.log(np.absolute(f1_freq)), cmap='gray')
ax2.imshow(np.log(np.absolute(f2_freq)), cmap='gray')

In [ ]:
f, (ax1, ax2) = plt.subplots(1, 2, sharey=True, figsize=(15, 15))
ax1.imshow(np.angle(f1_freq, deg=True), cmap='gray')
ax2.imshow(np.angle(f2_freq, deg=True), cmap='gray')

In [ ]:
ncps = f1_freq * np.conj(f2_freq) / np.abs(f1_freq * f2_freq)
shift = find_ift(ncps)

f, (ax1, ax2) = plt.subplots(1, 2, sharey=True, figsize=(15, 15))
ax1.imshow(np.absolute(ncps), cmap='gray')
ax2.imshow(np.absolute(shift), cmap='gray')

In [ ]:
ind = np.unravel_index(np.argmax(shift, axis=None), shift.shape)
[i - j for i, j in zip(shift.shape, ind)]

# Shifting Pink Noise 

In [ ]:
x_size = 256
y_size = 256

xx = np.linspace(-x_size / 2, x_size / 2, x_size)
yy = np.linspace(-y_size / 2, y_size / 2, y_size)

# Generating cloud image
whitenoise = np.random.normal(0, 1, (y_size, x_size))
ft_arr = find_ft(whitenoise)
kernel = freq_filter(xx, yy, factor=2.4)

pink_ft_arr = ft_arr * kernel
pink_noise = normalize_img(find_ift(pink_ft_arr).real)

# Shifting initial image
dx = 25
dy = 20

x_mesh, y_mesh = np.meshgrid(xx, yy)
shift = np.exp(-1j * 2 * np.pi  * (x_mesh * dx / y_size + y_mesh * dy / x_size));

# Apply the phase shift along both axes
f_freq_shifted = pink_ft_arr * shift
shifted_pink_noise = normalize_img(find_ift(f_freq_shifted).real)

show_images(pink_noise, shifted_pink_noise)

In [ ]:
%matplotlib notebook

# Generating cloud image
whitenoise = np.random.normal(0, 1, (y_size, x_size))
ft_arr = find_ft(whitenoise)
kernel = freq_filter(xx, yy, factor=2)

pink_ft_arr = ft_arr * kernel
pink_noise = find_ift(pink_ft_arr).real

# Shift values
dx = 200
dy = 0

# Shift values per one frame
fps = 30
seconds_num = 5
dxx = dx / fps / seconds_num
dyy = dy / fps / seconds_num

x_mesh, y_mesh = np.meshgrid(xx, yy)
shift = np.exp(-1j * 2 * np.pi  * (x_mesh * dxx / y_size + y_mesh * dyy / x_size));

snapshots_freq = [pink_ft_arr]
snapshots_spatial = [pink_noise]
for _ in range(fps * seconds_num):
    # print(np.sum(np.absolute(snapshots_freq[-1]) ** 2))
    snapshots_freq.append(snapshots_freq[-1] * shift)
    snapshots_spatial.append(find_ift(snapshots_freq[-1]).real)

    
fig = plt.figure(figsize=(8,8))
frame = snapshots_spatial[0]
im = plt.imshow(frame, interpolation='none', aspect='auto', cmap='gray')


def animate_func(i):
    im.set_array(snapshots_spatial[i])
    plt.title(f"X shift {i * dxx:.2f} \n Y shift {i * dyy:.2f}")
    return [im]


anim = animation.FuncAnimation(
    fig, 
    animate_func, 
    frames = seconds_num * fps,
    interval = 1000 / fps, # in ms
)

# Smooth transition

In [ ]:
%matplotlib inline

x_size = 256
y_size = 256

xx = np.linspace(-x_size / 2, x_size / 2, x_size)
yy = np.linspace(-y_size / 2, y_size / 2, y_size)  
depth = int(x_size * 0.3)

img_1 = gen_cloud(x_size, y_size)
img_2 = gen_cloud(x_size, y_size)

img_1_cut = img_1[:, -depth:x_size]
img_2_cut = img_2[:, 0:depth]
img_2_tr = img_2[:,depth:x_size]

x_kernel = spatial_smooth_filter(x_size, y_size, depth)
img1_cut_pr = img_1_cut * x_kernel + img_2_cut * (1 - x_kernel)
img_1[:,-depth:x_size] = img1_cut_pr
img_concat = np.concatenate((img_1, img_2_tr), axis=1)

show_images(img_1, img_2)

In [ ]:
plt.plot(range(depth), x_kernel[0])

In [ ]:
show_images(img_concat)

In [ ]:
img = gen_cloud(x_size, y_size)
img_new, add_img = make_img_transition_x(img, depth)
img_concat = np.concatenate((img_new, add_img), axis=1)

show_images(img, img_new, add_img, img_concat)

In [ ]:
img = gen_cloud(x_size, y_size)
img_new, add_img = make_img_transition_x(img, depth, is_dx_pos=False)
img_concat = np.concatenate((add_img, img_new), axis=1)

show_images(img, img_new, add_img, img_concat)

In [ ]:
img = gen_cloud(x_size, y_size)
img_new, add_img = make_img_transition_y(img, depth)
img_concat = np.concatenate((img_new, add_img), axis=0)

show_images(img, img_new, add_img, img_concat)

In [ ]:
img = gen_cloud(x_size, y_size)
img_new, add_img = make_img_transition_y(img, depth, is_dy_pos=False)
img_concat = np.concatenate((add_img, img_new), axis=0)

show_images(img, img_new, add_img, img_concat)

In [ ]:
img = gen_cloud(x_size, y_size)

img_new, img_x = make_img_transition_x(img, depth)
img_row_1 = np.concatenate((img_new, img_x), axis=1)

img_new, img_y = make_img_transition_y(img_row_1, depth)
img_row_2 = np.concatenate((img_new, img_y), axis=0)

show_images(img_row_1, img_row_2)

In [ ]:
img = gen_cloud(x_size, y_size)

img_new, img_x = make_img_transition_x(img, depth, is_dx_pos=False)
img_row_1 = np.concatenate((img_x, img_new), axis=1)

img_new, img_y = make_img_transition_y(img_row_1, depth)
img_row_2 = np.concatenate((img_new, img_y), axis=0)

show_images(img_row_1, img_row_2)

In [ ]:
img = gen_cloud(x_size, y_size)

img_new, img_x = make_img_transition_x(img, depth)
img_row_1 = np.concatenate((img_new, img_x), axis=1)

img_new, img_y = make_img_transition_y(img_row_1, depth, is_dy_pos=False)
img_row_2 = np.concatenate((img_y, img_new), axis=0)

show_images(img_row_1, img_row_2)

In [ ]:
img = gen_cloud(x_size, y_size)

img_new, img_x = make_img_transition_x(img, depth, is_dx_pos=False)
img_row_1 = np.concatenate((img_x, img_new), axis=1)

img_new, img_y = make_img_transition_y(img_row_1, depth, is_dy_pos=False)
img_row_2 = np.concatenate((img_y, img_new), axis=0)

show_images(img_row_1, img_row_2)

# Example of XY smooth transition and shift

In [ ]:
%matplotlib inline

x_size = 256
y_size = 256
depth = int(x_size * 0.3)

img = gen_cloud(x_size, y_size)

new_img = make_img_transition_xy(img, depth)
shifted_img = shift_img_xy(new_img, img.shape, 75, 50)

show_images(new_img)

In [ ]:
show_images(shifted_img)

In [ ]:
%matplotlib notebook

# Shift values
dx = 200
dy = 200

# Shift values per one frame
fps = 30
seconds_num = 5
dxx = dx / fps / seconds_num
dyy = dy / fps / seconds_num

dx_arr = [0]
dy_arr = [0]
for i in range(fps * seconds_num):
    dx_arr.append(dx_arr[-1] + dxx)
    dy_arr.append(dy_arr[-1] + dyy)
    

for i in range(fps * seconds_num):
    dx_arr[i] = round(dx_arr[i])
    dy_arr[i] = round(dy_arr[i])
    

x_mesh, y_mesh = np.meshgrid(xx, yy)
snapshots = []
for i in range(fps * seconds_num):
    snapshots.append(shift_img_xy(new_img, (256, 256), dx_arr[i], dy_arr[i]))

    
fig = plt.figure(figsize=(5,5))
frame = snapshots[0]
im = plt.imshow(frame, interpolation='none', aspect='auto', cmap='gray')


def animate_func(i):
    im.set_array(snapshots[i])
    plt.title(f"X shift {i * dxx:.2f} \n Y shift {i * dyy:.2f}")
    return [im]


anim = animation.FuncAnimation(
    fig, 
    animate_func, 
    frames = seconds_num * fps,
    interval = 1000 / fps, # in ms
)

In [ ]:
show_images(new_img)

# 2D distribution test

In [ ]:
import numpy as np
from scipy.optimize import minimize

# True parameters of the 2D exponential distribution
true_lambda1 = 0.5
true_lambda2 = 0.8

# Number of data points
n = 100

# Generate random data from the 2D exponential distribution
np.random.seed(42)
x = np.random.exponential(scale=1/true_lambda1, size=n)
y = np.random.exponential(scale=1/true_lambda2, size=n)

# Define the negative log-likelihood function
def neg_log_likelihood(params):
    lambda1, lambda2 = params
    return -np.sum(np.log(lambda1 * lambda2 * np.exp(-lambda1 * x - lambda2 * y)))

# Initial parameter guesses for optimization
initial_params = [0.1, 0.1]

# Maximize the negative log-likelihood to estimate parameters
result = minimize(neg_log_likelihood, initial_params, method='L-BFGS-B')

# Extract estimated parameters
estimated_lambda1, estimated_lambda2 = result.x

# Print true and estimated parameters
print("True Lambda 1:", true_lambda1)
print("Estimated Lambda 1:", estimated_lambda1)
print("True Lambda 2:", true_lambda2)
print("Estimated Lambda 2:", estimated_lambda2)


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Set the parameters for the bivariate exponential distribution
lambda1 = 0.5
lambda2 = 0.7

# Generate synthetic data from the bivariate exponential distribution
num_samples = 100
x_samples = np.random.exponential(scale=1/lambda1, size=num_samples)
y_samples = np.random.exponential(scale=1/lambda2, size=num_samples)

# Create a scatter plot of the generated data
plt.scatter(x_samples, y_samples, color='blue', alpha=0.5)
plt.title('2D Exponential Distribution')
plt.xlabel('X')
plt.ylabel('Y')
plt.grid(True)
plt.show()


In [ ]:
img = gen_cloud(100, 100)
img_freq = find_ft(img)
magnitude = np.absolute(img_freq)

plt.imshow(np.log(magnitude))

In [ ]:
import numpy as np
from scipy.optimize import minimize

def bivariate_exponential_log_likelihood(params, x, y, z):
    lambda1, lambda2 = params
    n = len(x)
    log_likelihood = n * np.log(lambda1 * lambda2) - lambda1 * np.sum(x) - lambda2 * np.sum(y)
    return -log_likelihood

# Convert the 2D z array into separate x, y, and z values
x_indices, y_indices = np.indices(magnitude.shape)
x = x_indices.flatten()
y = y_indices.flatten()
magnitude = magnitude.flatten()

initial_guess = [0.5, 0.5]  # Initial parameter guesses
result = minimize(bivariate_exponential_log_likelihood, initial_guess, args=(x, y, z), method='BFGS')
estimated_params = result.x

print("Estimated Parameters:", estimated_params)

In [ ]:
z = np.array([[0.1, 0.05, 0.2],
              [0.02, 0.15, 0.1],
              [0.05, 0.3, 0.25]])

x_indices, y_indices = np.indices(z.shape)
x = x_indices.flatten()
y = y_indices.flatten()
z = z.flatten()